In [1]:
import pandas as pd
import numpy as np
import nltk
import os
import nltk.corpus
import glob


In [2]:
from pathlib import Path
# change these to the directory where you cloned openvirusdev
HOME = str(Path.home())
OPENVIRUSDEV = HOME + "/" + "workspace/openvirusdev"
PROJECTS = HOME + "/" + "projects"


In [3]:
# PMR's projects. 
IPYNB = OPENVIRUSDEV + "/" + "ipynb"
VIRUS2019 = OPENVIRUSDEV + "/" + "viral2019"

OPENBATTERY = PROJECTS + "/" + "open-battery"
LIION = OPENBATTERY + "/" + "liion"



In [4]:
"""
import os
import sys
nb_dir = os.path.split(os.getcwd())[0]
# print("sys.path "+str(sys.path))
if nb_dir not in sys.path:
    sys.path.append(nb_dir)
"""

import amifiles as af
import amiwords as aw
import amistats as ast


[nltk_data] Downloading package punkt to /Users/pm286/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [5]:
# ==================
# from scikit-learn
# https://scikit-learn.org/stable/auto_examples/bicluster/plot_bicluster_newsgroups.html#sphx-glr-auto-examples-bicluster-plot-bicluster-newsgroups-py
# ==================

In [6]:
# MAIN ENTRY

project = VIRUS2019
# project = LIION
analyze_frequencies = False

os.chdir(project)

if analyze_frequencies:
    ast.analyze_frequencies_section(project, 'abstract')
    ast.analyze_frequencies_all(project);

files = af.extract_files(project, 'all')
print("files "+str(len(files)))
corpus = aw.extract_filtered_word_lists(files)

from sklearn.feature_extraction.text import TfidfVectorizer

print("corpus " + str(len(corpus)))
vectorizer = TfidfVectorizer(input='str')
text = vectorizer.fit_transform(corpus)
# print(vectorizer.get_feature_names())
print(text.shape)

cl.find_optimal_clusters(text, 20)


files 17144
corpus 17144


AttributeError: 'list' object has no attribute 'lower'